In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient

service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
wiki_index = os.getenv("AZURE_SEARCH_INDEX_WIKI")
video_index = os.getenv("AZURE_SEARCH_INDEX_VIDEO")
credential = DefaultAzureCredential()


In [3]:
wiki_search_client = SearchClient(service_endpoint, wiki_index, credential)
video_search_client = SearchClient(service_endpoint, video_index, credential)

In [9]:
results_wiki = wiki_search_client.search(search_text="register a PDM Link Server", top=5)
results_video = video_search_client.search(search_text="register a PDM Link Server", top=5)

In [10]:
for result_wiki, result_video in zip(results_wiki, results_video):
    print('result_wiki', result_wiki['@search.score'], 'result_video', result_video['@search.score'])

result_wiki 10.9183445 result_video 6.379058
result_wiki 7.562377 result_video 5.8741236
result_wiki 6.5556664 result_video 5.2683687
result_wiki 5.7498865 result_video 4.63564
result_wiki 5.033559 result_video 4.5281897


In [14]:
!azd auth login

Retrieving subscriptions...
Logged in to Azure.



To update to the latest version, run:
winget upgrade Microsoft.Azd


In [13]:
from langchain_openai import AzureChatOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from backend.auth.auth_utils import get_authenticated_user_details
from backend.security.ms_defender_utils import get_msdefender_user_json
from quart import request
# from llama_index.llms.azure_openai import AzureOpenAI

In [14]:
api_version = os.environ.get("AZURE_OPENAI_PREVIEW_API_VERSION")
aoai_api_key = os.environ.get("AZURE_OPENAI_KEY")
ad_token_provider = get_bearer_token_provider(DefaultAzureCredential(),"https://cognitiveservices.azure.com/.default")
endpoint = (
            os.environ.get("AZURE_OPENAI_ENDPOINT")
            if os.environ.get("AZURE_OPENAI_ENDPOINT")
            else f"https://{os.environ.get('AZURE_OPENAI_RESOURCE')}.openai.azure.com/"
        )
default_headers = {"x-ms-useragent": "GitHubSampleWebApp/AsyncAzureOpenAI/1.0.0"}


authenticated_user_details = get_authenticated_user_details({})
conversation_id = None     
user_json = get_msdefender_user_json(authenticated_user_details, {}, conversation_id)


# from llama_index.llms.azure_openai import AzureOpenAI

# llm = AzureOpenAI(
#     engine=os.environ.get("AZURE_OPENAI_MODEL"),
#     model=os.environ.get("AZURE_OPENAI_MODEL_NAME"),
#     api_key=aoai_api_key,
#     azure_endpoint=endpoint,
#     api_version=api_version,
#     AZURE_AD_TOKEN_PROVIDER = ad_token_provider,
#     use_azure_ad =True,
#     kwargs={'default_headers': default_headers, "user": user_json},
# )

llm = AzureChatOpenAI(
    deployment_name = os.environ.get("AZURE_OPENAI_MODEL"),
    openai_api_version = api_version,
    openai_api_key = aoai_api_key,
    azure_ad_token_provider = ad_token_provider,
    default_headers = default_headers,
    azure_endpoint = endpoint,
    include_response_headers = True,
    streaming= True,
    temperature= 0.15,
    # user_json = user_json
    # model_kwargs = {'': user_json}
    # logprobs=True,
    model_kwargs={"stream_options": {"include_usage": True}}
    )

In [45]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with colorful personality."
    ),
    ChatMessage(role="user", content="Hello"),
]

response = llm.chat(messages)
print(response)

assistant: Ahoy there, matey! Welcome aboard! What brings ye to these treacherous waters today? Lookin' for treasure, a tale, or perhaps a bit o' both? Arrr!


In [6]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that helps user.",
    ),
    ("human", "How are you?"),
]
ai_msg = llm.invoke(messages)
# apim_request_id = ai_msg.response_metadata['headers']['apim-request-id']
ai_msg


AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_80a1bad4c7'}, id='run-3f724057-5589-4b85-9fd8-311050c8c8b3-0', usage_metadata={'input_tokens': 24, 'output_tokens': 28, 'total_tokens': 52})

In [40]:
from backend.utils import (
    format_as_ndjson,
    format_stream_response,
    format_non_streaming_response,
    convert_to_pf_format,
    format_pf_non_streaming_response,
)